# Simulated annealing. Libraries.

## Simulated annealing

Let's consider cooling of metal which consists of freely moving atoms searching for position with minimal energy. Due to high temperature, they have quite big kinetic energy and can end up in totally different positions with different potential energies. However, as temperature decreases, kinetic energy decreases and atoms move to positions with higher potential energy with less probablity, which means that on average atoms are moving towards minimum. Since atoms had time to "explore" their surroundings they are very likely ending in position with minimal or close to minimal energy. <br>
Research of this physical phenomena in 80s resulted in computer modeling of this process that became base for simulated annealing algorithm. Now let's consider algorithm's pseudocode in Python, where we try to find **state** with minimal **energy**:
```python
s=s0 #initial state
for i in range(number_of_iterations):
    T=temperature(i)
    s_new=neighbour(s)
    if probability_of_transition(Energy(s),Energy(s_new),T)>random(0,1):
        s=s_new
        
print(s)
```
We can see 5 important facts:

#### Energy function

This function is key for solution which is defined explicitly only by problem we are trying to solve. Basically, it has input - state, which can be one number or number list and output - one number (energy) we are optimizing for. For example, in travelling salesman problem energy function is distance function whose input is sequence of cities' numbers - path. 

#### Iterative algorithm

You have to do a lot of iterations which will on average get state closer and closer to optimal. Overall, the more iterations you do the larger probability of getting needed solution. However, computing time always has to be considered.

#### Generation of a neighbour

In order to get better results, initial state has to move in coordinate space. This movement is achieved through generation random neighbour by slightly tweaking parametres of current state. For example, new path in travelling salesman problem can be generated by swapping two cities.

#### Probability of transition

Main idea behind effectiveness of simulated annealing algorithm is exploring coordinate space before making final decisions. In contrary, greedy algorithm will often fail in finding of global minimum because it always picks better state as it seems *right now*. However, we have to discourage (but not fully shutdown) picking worse states by introducing probability of picking worse state which is, obviously, less than 1. This probability has to get lower and lower as initial exploration is finished, which is achieved with help of temperature function. Now we can write how function probability_of_transition(Energy(s),Energy(s_new),T):
```python
def probability_of_transition(Energy_s,Energy_s_new,T):
    if (Energy_s_new<Energy_s):
        p=1 #we are moving to better state with 100% probability
    else:
        Delta_E=Energy_s_new-Energy_s #probability of transition depends on how "bad" is new state
        p=math.exp(-Delta_E/(Delta_E_Avg*T)) #this probability will be always less than 1
        
    return p
```
The most mysterious string of code is probably definition of probability in second case: $$p=e^{-\frac{\Delta E}{\Delta E_{avg}T}}$$ First of all, we calculate difference in energies between states, because, from physical point of view there is smaller probability that atom gets enough kinetic energy to transit to neigbour state. And from optimization point we cannot allow getting in not beneficial states very often because state can waste a lot of time returning back. Secondly, we have new value $\Delta E_{avg}$ which serves as standard value for coherence between early and late parts of optimization. Thirdly, we have temperature value which controls probability of picking with time. You can easily see that with lowering temperature, whole probability gets smaller. The same behaviour occurs in real world where with lowering temperature probability of getting high kinetic energy quickly reduces. Finally, exponent is took from Physics laws. Since numbers $\Delta E$,$\Delta E_{avg}$ and $T$ are positive, exponent is always between 0 and 1 plus behaviour of this function make it perfect candidate for probability calculation.

#### Temperature function

From physical analogy we introduce temperature or ability to freely move to positions with higher energy. This function is usually fully derived from 3 numbers: initial probability $p_0$ of transition with $\Delta E=\Delta E_{avg}$, final probability $p_n$ of transition with $\Delta E=\Delta E_{avg}$ and number of iterations $n$. From definition of transition probability initial and final temperatures are equal to $T_0=-1/\ln(p_0)$ and $T_n=-1/\ln(p_n)$, respectively. Then we introduce the rate of decay $f=\sqrt[n]{\frac{T_n}{T_0}}$, which helps simply recursively calculate temperature for current iteration as $T_i=fT_{i-1}$:

In [ ]:
import math #information about math library is in the second notebook

n=10
pn=0.001    #defining 3 parametres
p0=0.5

T0=-1/math.log(p0) #math.log(x) is natural logarithm ln(x)
Tn=-1/math.log(pn)
f=(Tn/T0)**(1/n)   

print('Initial temperature:',T0)
print('Final temperature:',Tn)
print('Rate of decay:',f)

T=T0 #start of cycle
for i in range(n):
    T=T*f
    print('Iteration',str(i+1),' Temperature:',str(T))

## Application in traveling salesman problem

Firstly, we rewrite helping functions from previous lesson:

In [ ]:
def create_map(n,seed):
    import matplotlib.pyplot as plt
    import random
    random.seed(seed) #seed of random number generator, it allows to get the same map everytime

    #n is a number of cities
    x=[] #list of x coordinates of cities (in km)
    y=[] #list of y coordinates of cities (in km)
    names=[] #list of cities' names

    #now we create loop to append coordinates to lists
    for i in range(n):
        x.append(random.randint(0,1000))
        y.append(random.randint(0,1000))
        names.append(chr(65+i)) #we name cities as A,B,C...


    #finally, we plot map
    plt.figure(figsize=(7,7))
    plt.scatter(x,y)
    for i in range(n): plt.text(x[i]+10,y[i]-20,names[i]) #adding names of cities
    plt.show()
    return x,y,names #returns coordinates of cities

In [ ]:
def distance_dictionary(x,y):  
    n=len(x) #number of cities
    cities={} #dictionary which stores all possible distances between two cities
    for i in range(n):
        city={} #we create small dictionary which includes distances from city i to all other cities
        for k in range(n):
            if (i!=k): 
                city[k]=((x[i]-x[k])**2+(y[i]-y[k])**2)**(1/2) #calculating distance

        cities[i]=city

    return cities

In [ ]:
m=45 #number of cities
x,y,names=create_map(m,78)
cities=distance_dictionary(x,y)

Now we create random path - randomly shuffled list of numbers from 0 to n-1 plus first element (because path is closed by definition) and defining function distance, which, basically, is energy function.

In [ ]:
import random

s=list(range(m)) #creating path 0-1-2-...
random.shuffle(s) #shuffling it
print(s)

In [ ]:
def distance(path,cities):
    d=0
    for i in range(len(path)-1): d+=cities[path[i]][path[i+1]]
    d+=cities[path[len(path)-1]][path[0]]
    return d        

In [ ]:
#defining parametres
n=1000000
pn=1E-6  
p0=0.5

T0=-1/math.log(p0)
Tn=-1/math.log(pn)
f=(Tn/T0)**(1/n)   

T=T0 
for i in range(n):
    if (T<Tn): break
    T=T*f    
    l = random.randint(2, m-1)
    k = random.randint(0, m-l)
    s_new=s[:]
    s_new[k:(k+l)] = reversed(s_new[k:(k+l)]) #swapping part of s to get s_new
    
    d=distance(s,cities)
    d_new=distance(s_new,cities)
    delta_d=math.fabs(d_new-d) #we always calculate delta_d for calculating delta_d_avg
    if (i==0): 
        delta_d_avg=delta_d #assign delta_d_avg when its equal 0
    else:
        delta_d_avg=(i*delta_d_avg+delta_d)/(i+1) #changing delta_d_avg  
    #probability_of_transition function    
    if (d_new<=d):
        p=1
    else: 
        p=math.exp(-delta_d/(delta_d_avg*T))
    if (p>=random.random()): s=s_new[:]
        
    if (i%(n//10)==0): print(d) #print distance 10 times during cycle

Now we plot found path on the map

In [ ]:
#After we have found optimal or close to optimal path, we can show it on map
def path_on_map(route,x,y,names,cities):
    import matplotlib.pyplot as plt
    n=len(names) #number of cities
    optimal_x=[] #consecutive coordinates of cities from the best route
    optimal_y=[]
    for i in range(n): 
        optimal_x.append(x[s[i]])
        optimal_y.append(y[s[i]])

    optimal_x.append(optimal_x[0])    
    optimal_y.append(optimal_y[0])  

    plt.figure(figsize=(7,7))    
    plt.plot(optimal_x,optimal_y,'r')
    plt.scatter(x,y)
    for i in range(n): plt.text(x[i]+10,y[i]-20,names[i]) #adding names of cities
    plt.title("Distance: "+str(int(distance(s,cities)))+" km")
    plt.show()

In [ ]:
path_on_map(s,x,y,names,cities)

As you can see, the path is pretty close to optimal. If you want better result, you should play with 3 parametres we defined. And that is probalby the major problem with simulated annealing - you have to specially tune these numbers (which later will be called *hyperparametres*) for each problem you are solving. However, the result is much better than in case of greedy algorithm. Moreover, simulated annealing has one more huge advantage over greedy algorithm - it can work with continuous functions.

## Application in optimization of continuous functions

Let's optimize function $f(x)$ of 2 real variables: $$f(x)=\frac{1}{5}+x^2+y^2-\frac{1}{10}\cos(6\pi x)-\frac{1}{10}\cos(6\pi y)$$

In [ ]:
def f(x,y):
    return 0.2 + x**2 + y**2 - 0.1*np.cos(6.0*3.1415*x) - 0.1*np.cos(6.0*3.1415*y)

Then we import several libraries for much easier work and representation of results.

In [ ]:
import numpy as np #library for work with numeric lists - we are going to learn about it in several lessons
import matplotlib.pyplot as plt #builder of graphs
import matplotlib.animation as an #creator of animations
from IPython.display import HTML #HTML player of animations

In [ ]:
#Now we can plot color graph of function (blue means smaller values and yellow - larger ones)
a=1
na=200
x=np.linspace(-a,a,na)
y=np.linspace(-a,a,na).reshape(-1, 1)
fig=plt.figure(figsize=(7,7))
ax=plt.subplot(111)
im = ax.imshow(f(x,y),cmap='viridis',animated=True)
plt.show()

As you can see, this graph naturally has a lot of minimums and maximums. This is the perfect test of searching one global minimum for simulated annealing. Next we are defining hyperparametres of algorithm and run it. Algorithm hasn't changed much except energy function which is now, obviously, equal to $f(x)$. In parallel, we are writing down values (x,y) or current state for later creating animation of searching for minimum.

In [ ]:
#Source: http://apmonitor.com/me575/index.php/Main/SimulatedAnnealing
#Size of boundaries
a=1
# Number of cycles
n = 2500
# Number of accepted solutions
na = 0.0
# Probability of accepting worse solution at the start
p1 = 0.7
# Probability of accepting worse solution at the end
pn = 0.001
# Initial temperature
t1 = -1.0/np.log(p1)
# Final temperature
tn = -1.0/np.log(pn)
# Fractional reduction every cycle
frac = (tn/t1)**(1.0/(n-1.0))
# Initialize x and y - history of points
x=np.zeros(n+1)
y=np.zeros(n+1)
x[0]=0.7
y[0]=0.8
#The best value of the function
fc=f(x[0],y[0])
# DeltaE Average
DeltaE_avg = 0.0
#Main point and experimental point
xi=x[0]
yi=y[0]
xc=x[0]
yc=y[0]
print(xi,yi,frac)

In [ ]:
t=t1 #Initial temperature
for i in range(n):     
    # Generate new point
    xi=xc+a*(np.random.rand() - 0.5)
    yi=yc+a*(np.random.rand() - 0.5)    
    # Clip to upper and lower bounds 
    #There is a chance that point can randomly get out of boundaries 
    #In that case we return point on the edge of boundaries using following code
    xi = max(min(xi,a),-a)
    yi = max(min(yi,a),-a)   
    DeltaE = abs(f(xi,yi)-fc)
    if (f(xi,yi)>fc):
        # Initialize DeltaE_avg if a worse solution was found
        # on the first iteration
        if (i==0): DeltaE_avg = DeltaE
        # objective function is worse
        # generate probability of acceptance
        p = np.exp(-DeltaE/(DeltaE_avg * t))        
        # determine whether to accept worse point
        if (np.random.rand()<p):
            # accept the worse solution
            accept = True
        else:
            # don't accept the worse solution
            accept = False
    else:
        # objective function is lower, automatically accept
        accept = True
    if (accept==True): 
        #update value of function
        xc=xi
        yc=yi
        fc = f(xi,yi)        
        # increment number of accepted solutions
        na = na + 1.0
        # update DeltaE_avg
        DeltaE_avg = (DeltaE_avg * (na-1.0) +  DeltaE) / na
    # Record the best x and y values at the end of every cycle    
    x[i+1] = xc
    y[i+1] = yc   
    # Lower the temperature for next cycle
    t = frac * t

Now we can define one more function for creating animation and draw it:

In [ ]:
def updatefig(*args):
    global i
    i+=1
    ind=i*24
    xx=na*(1+x[ind]/a)/2
    yy=na*(1+y[ind]/a)/2
    line.set_data([xx],[yy])     
    return line,

In [ ]:
a=1
na=200
x_image=np.linspace(-a,a,na)
y_image=np.linspace(-a,a,na).reshape(-1, 1)
fig=plt.figure(figsize=(7,7))
ax=plt.subplot(111)
im = ax.imshow(f(x_image,y_image),cmap='viridis',animated=True)
line=ax.plot([50], [50], 'ro',markersize=13,animated=True)[0]
i=-1
ani = an.FuncAnimation(fig, updatefig, interval=300, blit=True, frames=90)
HTML(ani.to_html5_video())

You can clearly see that first red ball, which represents current state randomly wanders around, but as temperature lowers it starts to spend more time near center or, at least dark blue zone. Thus, simulated annealing has broad application in solving optimization problems and its major advantage is the ability to find global minimum. However, it is quite hard to get result with desired accuracy and sometimes a lot of the same experiments had to be done just to choose the best result.

## Practice task

Download file opt_lib.py and write there function sim_ann(energy,neighbour,hyperp,s0) where energy - energy function, neighbour - function which creates neighbour state of current state, hyperp - list of form [p0,pn,n], where p0 - initial probability, pn - final probability, n - number of iterations and s0 - initial state (a list of parametres). This function has to return s - optimal state after n iterations. <br>
**Hint:** Yes! Functions can be inputs too! Take a look at this example:

In [ ]:
def f(x):
    return x**2

def print_function(f,x):
    print('f({0})={1}'.format(x,f(x))) #similar format of a string as in C#

In [ ]:
print_function(f,4)

In [ ]:
#We can change function and it will work!
#The only restriction is that function f(x) has to have 1 argument
def f(x):
    return 2

print_function(f,4)

Use created module to optimize function:

In [ ]:
def f(a,b):
    return (1*a+b-4)**2+(2*a+b-5)**2+(3*a+b-7)**2+(4*a+b-8)**2